In [1]:
import pandas as pd
import numpy as np
from Lib import Algen_lib as lib

# Load Data

In [2]:
#Data Latih
df = pd.read_excel('data/sms_clean_norm_+_new_data_3.xlsx')

corpusA = df.sms.tolist()
labelA = df.label.tolist()
len(df)

1143

# Ekstraksi Fitur

In [3]:
fitur = lib.feature_extraction(corpusA)

# Seleksi Fitur Menggunaka Algoritma Genetika

In [ ]:
jumlah_populasi = 200
jumlah_point = 1000 #jumlah titik point crossover
prob_mutasi = 0.2 #probabilitas permutasi

K=30

metode = "tfidf"

alpha = 1
target = 0.99 #target akurasi

panjang_fitur = len(fitur)

features_1_bin = [1 for i in range(panjang_fitur)]
pop = lib.create_population(jumlah_populasi-1, panjang_fitur)
pop.append(features_1_bin)

fns_1 = lib.fitness_kf(corpusA, labelA, features_1_bin, fitur, alpha = alpha, metode = metode, K=K)
print("fitness all features:",fns_1)

fitnes_pop = list()
# temp_features_fitness = list()
for features_bin in pop:
    fns = lib.fitness_kf(corpusA, labelA, features_bin, fitur, alpha = alpha, metode = metode, K=K)
    fitnes_pop.append(fns)
#     temp_features_fitness.append([fns, features_bin])
# fitnes_pop.append(fns_1)

print(len(fitnes_pop),"|",len(pop))

pop_used = dict()
popp_0 = list(pop)
for p, fit in zip(popp_0, fitnes_pop):
    bin_str = lib.int_to_str(p) 
    pop_used.update({bin_str:fit})

# print("")
    
# print(pop[0])

best_fitur_list = list()
best_fitness_list = list()
best_generasi_list = list()

best_jumlah_fitur_list = list()

# save_aja = list()

gen=0
while True:
    gen+=1
    print("-"*25)
    print("Generasi ke",gen , "[Terbaik]")
    rw = lib.get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = lib.select_parents(rw)

    best_fitur_list_per_generasi = list()
    best_fitness_list_per_generasi = list()
    for p in parents:
        best_generasi_list.append(gen)
        family_fitness=list()
        family_fitur=list()

        mama_index = p[0]
        papa_index = p[1]
        
        m4ms = fitnes_pop[mama_index]
        family_fitness.append(m4ms)
        m4mr = list(pop[mama_index])
        family_fitur.append(m4mr)
#         best_jumlah_fitur_list.append(sum(m4mr))
        
        p4ps = fitnes_pop[papa_index]
        family_fitness.append(p4ps)
        p4pr = list(pop[papa_index])
        family_fitur.append(p4pr)
#         best_jumlah_fitur_list.append(sum(p4pr))
        
        #crossover dan mutasi
        bin_mama = list(pop[mama_index])
        bin_papa = list(pop[papa_index])
        anak_binary = lib.crossover(bin_mama, bin_papa, panjang_fitur,  jumlah_titik = jumlah_point, prob_mutasi = prob_mutasi)
        
        #mencari fitness untuk child
        a1 = anak_binary[0]
        a2 = anak_binary[1]
        a3 = anak_binary[2]
#         print(a2)
        
        if sum(a1)<=0:
            a1[-1]=1
            a1[-2]=1
        if sum(a2)<=0:
            a2[-1]=1
            a2[-2]=1
        if sum(a3)<=0:
            a3[-1]=1
            a3[-2]=1

        str_a1 = lib.int_to_str(a1)
        if str_a1 in pop_used:
            family_fitur.append(a1)
            family_fitness.append(pop_used[str_a1])
#             best_jumlah_fitur_list.append(sum(a1))
            
        else:
            family_fitur.append(a1)
            fitness_anak_1 = lib.fitness_kf(corpusA, labelA, a1, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_1)
            pop_used.update({str_a1:fitness_anak_1})
#             best_jumlah_fitur_list.append(sum(a1))
            
        str_a2 = lib.int_to_str(a2)
        if str_a2 in pop_used:
            family_fitur.append(a2)
            family_fitness.append(pop_used[str_a2])
#             best_jumlah_fitur_list.append(sum(a2))
        else:
            family_fitur.append(a2)
            fitness_anak_2 = lib.fitness_kf(corpusA, labelA, a2, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_2)
            pop_used.update({str_a2:fitness_anak_2})
#             best_jumlah_fitur_list.append(sum(a2))
            
        str_a3 = lib.int_to_str(a3)
        if str_a3 in pop_used:
            family_fitur.append(a3)
            family_fitness.append(pop_used[str_a3])
#             best_jumlah_fitur_list.append(sum(a3))
        else:
            family_fitur.append(a3)
            fitness_anak_3 = lib.fitness_kf(corpusA, labelA, a3, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_3)
            pop_used.update({str_a3:fitness_anak_3})
#             best_jumlah_fitur_list.append(sum(a3))

        
        #mencari fitness terbaik untuk satu keluarga
        best_family_fitness = max(family_fitness)
        bf_index = family_fitness.index(best_family_fitness)
        best_family_fitur = family_fitur[bf_index]

        best_fitness_list.append(best_family_fitness)
        best_fitur_list.append(best_family_fitur)

        best_fitness_list_per_generasi.append(best_family_fitness)
        best_fitur_list_per_generasi.append(best_family_fitur)
        best_jumlah_fitur_list.append(sum(best_family_fitur))

        print("   ",best_family_fitness)
        print("   ","."*30,"|",str(sum(best_family_fitur))+"/"+str(len(best_family_fitur)),"|",panjang_fitur,"|",gen)

    pop = list(best_fitur_list_per_generasi)
#     pop.append(features_1_bin)
    fitnes_pop = list(best_fitness_list_per_generasi)
#     fitnes_pop.append(fns_1)

    if len(pop)<2 or max(best_fitness_list)>=target:
        print("="*70)
        print("Best",max(best_fitness_list))
#         print("Generasi ke-",best_generasi_list[best_fitness_list.index(max(best_fitness_list))])
        good_fitur = best_fitur_list[best_fitness_list.index(max(best_fitness_list))]
        best_fitness_list.index(max(best_fitness_list))
        print('jumlah fitur     ', sum(good_fitur))
        print('jumlah fitur asli', panjang_fitur)
        print('good_fitur')
        break

fitness all features: 0.9353126405757984
200 | 200
-------------------------
Generasi ke 1 [Terbaik]
    0.9282501124606389
    .............................. | 2217/4378 | 4378 | 1
    0.9309041835357624
    .............................. | 3297/4378 | 4378 | 1
    0.9282950967161493
    .............................. | 3274/4378 | 4378 | 1
    0.9247863247863247
    .............................. | 3287/4378 | 4378 | 1
    0.9230094466936573
    .............................. | 3318/4378 | 4378 | 1
    0.9160143949617633
    .............................. | 2176/4378 | 4378 | 1
    0.9353126405757984
    .............................. | 4378/4378 | 4378 | 1
    0.9247188484030587
    .............................. | 3257/4378 | 4378 | 1
    0.9265407107512371
    .............................. | 3339/4378 | 4378 | 1
    0.9300269905533063
    .............................. | 3283/4378 | 4378 | 1
    0.9309266756635177
    .............................. | 3246/4378 | 4378 | 1
    0.92

In [5]:
#Testing
hasil_seleksi_fitur = good_fitur
lib.fitness_kf(corpusA, labelA, hasil_seleksi_fitur, fitur, alpha = alpha, metode = metode, K=K)

0.9361223571749886

In [6]:
for i in np.array(fitur)[lib.get_index(hasil_seleksi_fitur)]:
    print(i, end=", ")

0, 00, 000, 000untuk, 001, 0016282190254447, 0016282336033777, 0016285286552555, 002218, 0035, 006474, 01, 0100, 01123190427523627, 018801, 02, 021, 02129657333, 02136146799, 02152971540, 0215649770, 0215891174, 02171790989, 02194521396, 02199545139, 022, 0226, 0231327xxx, 024, 0248147033, 0253, 02599875, 02749447574, 0277285682, 0282566132, 03, 03299757, 0385, 04, 05, 06, 07, 076, 08, 0809, 081215008228, 081218844557, 081223052854, 0812330xxxx, 081233303888, 081281108799, 0812xxxxxxx, 081310537888, 081314447669, 081315277257, 081316005639, 081316523061, 081563257257, 081571xxx, 08157xxx, 081584650877, 081586769255, 0816288602, 0818433247, 0818643929, 081931753240, 082, 0821, 082110001021, 082113720777, 082122483312, 082127377377, 082134976289, 082189983017, 082193609797, 0822, 0823, 082310204777, 082319733334, 082325557768, 082327327705, 082328823345, 082329676767, 082329969711, 082374910777, 082375271777, 082375833300, 083139195872, 085104894998, 0852, 085215126428, 085216181177, 085

mie, migrasi, miitra, mikirin, miliard, million, milyar, min, minat, mineral, minggu, minggu2, mini, minimal, minimum, mining, minna, minta, minum, miring, misal, misi, mjd, mkios, mkios17, mkiosjkt, mksh, ml, mla, mlm, mmber, mnctv, mndpt, mngenai, mnghubungi, mningkat, mnta, mnyediakn, mnyesuaikan, mo, moal, mobcom, mobil, modal, model, modem, modus, mohon, more, motor, movie, mp, mraih, mrasa, msalah, msalh, msceis, msh, msk, mthr, mtronik, mu, mubazir, muda, mudah, mudah2an, muka, mul, mulai, muliani, muliyadi, multiple, mulu, mumpung, muncul, mungkin, murah, murdoyo, murtiani, music, musik, my, mycareply, mytsel1, myvplus, n, n091, n092, nabung, nagamas, nah, naha, naik, nak, naldisuparman, nama1, namakuuu, nambah, nang, nanggung, nanya2, nanyain, narti, nasabah, nasi, nasib, nb, ndisek, ndpt, ndut, negara, nego, negosiasi, nelp, nelpon, nemu, nenek, netmax, new, nf, ngajak, ngajakan, ngajakin, ngaku, ngambil, ngampus, ngantor, ngaret, ngasih, ngasihinnya, nge, ngebatin, ngebayang

In [7]:
fitur_pilihan = np.array(fitur)[lib.get_index(good_fitur)]
print(len(fitur_pilihan))
fitur_pilihan

3276


array(['0', '00', '000', ..., 'z10', 'zarkasi', 'zona'], dtype='<U26')

In [8]:

dict_data = {
    "fitness":best_fitness_list,
    "jumlah_fitur":best_jumlah_fitur_list,
    "generasi":best_generasi_list,
}

df_hasil = pd.DataFrame.from_dict(dict_data)
df_hasil


,fitness,jumlah_fitur,generasi
0,0.917814,3300,1
1,0.923077,3258,1
2,0.934435,3256,1
3,0.928273,3298,1
4,0.920355,3286,1
5,0.919568,3314,1
6,0.923032,3322,1
7,0.922987,3304,1
8,0.922987,3260,1
9,0.915070,2131,1


In [ ]:
print(len(best_fitness_list))

In [ ]:
cek = 0.93

grouped = df_hasil.groupby('generasi')
grouped.filter(lambda x: x['fitness'].mean() > cek)